In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samanantar/final_data/en-ta/train.ta
/kaggle/input/samanantar/final_data/en-ta/train.en
/kaggle/input/samanantar/final_data/en-ml/train.ml
/kaggle/input/samanantar/final_data/en-ml/train.en
/kaggle/input/samanantar/final_data/en-as/train.as
/kaggle/input/samanantar/final_data/en-as/train.en
/kaggle/input/samanantar/final_data/en-kn/train.kn
/kaggle/input/samanantar/final_data/en-kn/train.en
/kaggle/input/samanantar/final_data/en-pa/train.pa
/kaggle/input/samanantar/final_data/en-pa/train.en
/kaggle/input/samanantar/final_data/en-mr/train.mr
/kaggle/input/samanantar/final_data/en-mr/train.en
/kaggle/input/samanantar/final_data/en-hi/train.hi
/kaggle/input/samanantar/final_data/en-hi/train.en
/kaggle/input/samanantar/final_data/en-bn/train.bn
/kaggle/input/samanantar/final_data/en-bn/train.en
/kaggle/input/samanantar/final_data/en-te/train.te
/kaggle/input/samanantar/final_data/en-te/train.en
/kaggle/input/samanantar/final_data/en-or/train.or
/kaggle/input/samanantar/final_

In [83]:
import torch
import torch.nn as nn
import math
import numpy as np
import torch.nn.functional as F

In [84]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [85]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
print(f"Using device: {device}")


Using device: cuda


In [86]:
English_file = "/kaggle/input/samanantar/final_data/en-gu/train.en"
Gujarati_file = "/kaggle/input/samanantar/final_data/en-gu/train.gu"

In [87]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

In [88]:

# Gujarati letters and symbols
gujarati_letters = [
    'અ', 'આ', 'ઇ', 'ઈ', 'ઉ', 'ઊ', 'ઋ', 'ઌ', 'એ', 'ઐ', 'ઓ', 'ઔ', 'ક', 'ખ', 'ગ', 'ઘ', 'ઙ', 'ચ', 'છ', 'જ', 'ઝ', 'ઞ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન',
    'પ', 'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'ળ', 'વ', 'શ', 'ષ', 'સ', 'હ', 'ા', 'િ', 'ી', 'ુ', 'ૂ', 'ૃ', 'ૄ', 'ૅ', 'ે', 'ૈ', 'ો', 'ૌ', 'ૐ', 'ૡ', 'ૢ', '૱',
    'ં', 'ઃ', 'ૠ', 'ૉ', '૰', '્', 'ઁ', 'ઍ', '઼', 'ઑ'
]

# Gujarati numerals
gujarati_numerals = [chr(code) for code in range(0x0AE6, 0x0AEF + 1)]
print(gujarati_numerals)
# Additional symbols and characters
# additional_symbols = [
#     '”', '“', '‘', '~', ']', '\\', '»', '@', '❑', '`', '}', '|', '–', '_', '’', '।', '्', '©', '½', '[', '°', '₹', '{', '…'
# ]

# Common symbols and punctuation used in Gujarati text
other_symbols = [
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',

    ':', '<', '=', '>', '?'
]

# Combine all characters into one vocabulary list
Gujarati_vocab = [START_TOKEN] + gujarati_letters + gujarati_numerals + additional_symbols + other_symbols + [PADDING_TOKEN, END_TOKEN]

# Example usage
print("Gujarati Vocabulary:")
print(Gujarati_vocab)
print(len(Gujarati_vocab))
print(len(set(Gujarati_vocab)))


['૦', '૧', '૨', '૩', '૪', '૫', '૬', '૭', '૮', '૯']
Gujarati Vocabulary:
['<START>', 'અ', 'આ', 'ઇ', 'ઈ', 'ઉ', 'ઊ', 'ઋ', 'ઌ', 'એ', 'ઐ', 'ઓ', 'ઔ', 'ક', 'ખ', 'ગ', 'ઘ', 'ઙ', 'ચ', 'છ', 'જ', 'ઝ', 'ઞ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન', 'પ', 'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'ળ', 'વ', 'શ', 'ષ', 'સ', 'હ', 'ા', 'િ', 'ી', 'ુ', 'ૂ', 'ૃ', 'ૄ', 'ૅ', 'ે', 'ૈ', 'ો', 'ૌ', 'ૐ', 'ૡ', 'ૢ', '૱', 'ં', 'ઃ', 'ૠ', 'ૉ', '૰', '્', 'ઁ', 'ઍ', '઼', 'ઑ', '૦', '૧', '૨', '૩', '૪', '૫', '૬', '૭', '૮', '૯', '”', '“', '‘', '~', ']', '\\', '»', '@', '❑', '`', '}', '|', '–', '_', '’', '।', '्', '©', '½', '[', '°', '₹', '{', '…', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '<PADDING>', '<END>']
140
140


In [89]:
d = set()

for i in Gujarati_vocab:
    if i not in d:
        d.add(i)
    else:
        print(i)

In [90]:
English_vocab = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', '<', '=', '>', '?', '@',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
    'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
    'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
    'y', 'z',
    PADDING_TOKEN, END_TOKEN
] 
 
# Print the length of English vocabulary
print("English vocabulary:", (English_vocab))

English vocabulary: ['<START>', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '<PADDING>', '<END>']


In [91]:
with open(Gujarati_file, 'r') as file:
    gujarati_sentences = file.readlines()
with open(English_file, 'r' ) as file:
    english_sentences = file.readlines()

In [92]:
print(len(gujarati_sentences))
print(len(english_sentences))

3054273
3054273


In [93]:
for i,j  in zip(gujarati_sentences[:10],english_sentences[:10]):
    print(i)
    print(j)

ઘણા જાતજાતની માહિતી અને અફવા ફેલાવે છે.

But many have tried to supply missing details.

જૈશ-એ-મોહમ્મદના કંટ્રોલ રૂમ પણ નષ્ટ થઈ ગયા છે.

Jaish-e-Mohammed (JeM) control rooms were also reportedly destroyed.

તેમણે આ અંગે વધુ વાત કરવાનો ઇનકાર કર્યો.

He declined to speak further on the issue.

હરિયાણામાં 85 લાખ મહિલા અને ટ્રાન્સઝેન્ડર મતદારો સહિત કુલ 1.83 કરોડ મતદાતાઓ નોંધાયેલ છે

In Haryana, there are over 1.83 crore voters, out of which 83 lakh are women

તેથી તે કાળજીપૂર્વક કામ કરવા માટે જરૂરી છે.

Therefore, this has to be done cautiously.

સાથોસાથ ભારતીય જનતા પાર્ટીના યુવા મોરચાના રાષ્ટ્રીય અધ્યક્ષ પદે તેની તાજપોશી થાય એ માટે પણ દાવ ખેલ્યો હતો.

He was also the national vice-president of the All India Bharatiya Janata Yuva Morcha, the youth wing of the BJP.

"આ રેલીમાં મોદીએ કહ્યું, ""અહીંયા કુદરતી સ્ત્રોતોમાં થઇ રહેલા ઘટાડાની કોંગ્રેસને કોઇ પરવા ."

Congress is not bothered about the depletion of natural resources here, Modi said.

માતાનો તેમને વિગતવાર સમજવા માટે પ્રયાસ કરીએ.

Lets

In [94]:
index_to_gujarati = {k:v for k,v in enumerate(Gujarati_vocab)}
gujarati_to_index = {v:k for k,v in enumerate(Gujarati_vocab)}
index_to_english = {k:v for k,v in enumerate(English_vocab)}
english_to_index = {v:k for k,v in enumerate(English_vocab)}

In [95]:
total_sentences = 300000
english_sntcs = english_sentences[:total_sentences]
gujarati_sntcs = gujarati_sentences[:total_sentences]
english_sntcs = [sentence.rstrip('\n') for sentence in english_sntcs]
gujarati_sntcs = [sentence.rstrip('\n') for sentence in gujarati_sntcs]

In [96]:
max_seq_len = 200
def is_valid_token(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence,max_seq_len):
    return len(list(sentence)) <= (max_seq_len - 2) # need to add <START> and <END> tokens

valid_sntcs_idx = []
for index in range(len(gujarati_sntcs)):
    guj_sntc , eng_sntc = gujarati_sntcs[index], english_sntcs[index]
    if is_valid_length(guj_sntc, max_seq_len) \
        and is_valid_length(eng_sntc, max_seq_len) \
        and is_valid_token(guj_sntc, Gujarati_vocab) \
        and is_valid_token(eng_sntc, English_vocab):
        
        valid_sntcs_idx.append(index)
        
print(len(valid_sntcs_idx))

279642


In [97]:
english_sntcs = [english_sntcs[i] for i in valid_sntcs_idx]
gujarati_sntcs = [gujarati_sntcs[i] for i in valid_sntcs_idx]

In [98]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    
    def __init__(self, english_sntcs, gujarati_sntcs):
        self.english_sntcs = english_sntcs
        self.gujarati_sntcs = gujarati_sntcs
        
    def __len__(self):
        return len(self.english_sntcs)
    
    def __getitem__(self,idx):
        return self.english_sntcs[idx], self.gujarati_sntcs[idx]

In [99]:
dataset = TextDataset(english_sntcs, gujarati_sntcs)
len(dataset)

279642

In [100]:
import torch

d_model = 512
batch_size = 32
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_seq_len = 200
guj_vocab_size = len(Gujarati_vocab)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_seq_len,
                          guj_vocab_size,
                          english_to_index,
                          gujarati_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [101]:
train_loader = DataLoader(dataset, batch_size)


In [102]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=gujarati_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)


In [103]:
NEG_INFTY = -1e9

def create_masks(eng_batch, guj_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_seq_len, max_seq_len] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_seq_len, max_seq_len] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_seq_len, max_seq_len] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_seq_len, max_seq_len] , False)

    for idx in range(num_sentences):
        eng_sentence_length, guj_sentence_length = len(eng_batch[idx]), len(guj_batch[idx])
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_seq_len)
        guj_chars_to_padding_mask = np.arange(guj_sentence_length + 1, max_seq_len)
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, guj_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, guj_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, guj_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
#     print(f"encoder_self_attention_mask {encoder_self_attention_mask.size()}: {encoder_self_attention_mask[0, :10, :10]}")
#     print(f"decoder_self_attention_mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :10, :10]}")
#     print(f"decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :10, :10]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask
     

In [104]:
%%time
transformer.train()
print(device)
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, guj_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, guj_batch)
        optim.zero_grad()
        guj_predictions = transformer(eng_batch,
                                     guj_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(guj_batch, start_token=False, end_token=True)
        loss = criterian(
            guj_predictions.view(-1, guj_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == gujarati_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"gujarati Translation: {guj_batch[0]}")
            guj_sentence_predicted = torch.argmax(guj_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in guj_sentence_predicted:
                if idx == gujarati_to_index[END_TOKEN]:
                    break
                predicted_sentence += index_to_gujarati[idx.item()]
            print(f"gujarati Prediction: {predicted_sentence}")


            transformer.eval()
            guj_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_seq_len):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, guj_sentence)
                predictions = transformer(eng_sentence,
                                          guj_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_gujarati[next_token_index]
                guj_sentence = (guj_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break
            
            print(f"Evaluation translation (should we go to the mall?) : {guj_sentence}")
            print("-------------------------------------------")
     

cuda
Epoch 0
Iteration 0 : 5.799540996551514
English: But many have tried to supply missing details.
gujarati Translation: ઘણા જાતજાતની માહિતી અને અફવા ફેલાવે છે.
gujarati Prediction: થતઝ=ૉૉ00ૉ0@=ઢધ00ી000000ધ0@,@તત઼<°~ૐૐ~ત~–ૅ❑<<_ુ_<__1(((ઈ–ઝ***(ઑ(ઝ((ઑઑઑ(ઝઝ(ઝઑ__ધ(<સસસસસઑસસસસસસસ૮૮સ_સસસઑઈઈઈઈઈળ૮૮૮ળપધધ૮૮ઝળઝ5દ6થ૮ઑધથથથથથથથથથદ૦ધ૦ધથથથથથધ૦થથદ’ઢ’’’’’’’*ઢઢઢળ**ઢ*!ઢથહથ!થ!!!!!!$$!!$પએ!₹₹!!!!!!₹!!
Evaluation translation (should we go to the mall?) : ('        ીી               ,,,ાકાાાૅૅૐ્્્ૅૅૅૅ્્્?___      *((((( ા(((ઑઝઝસસસસસધધધસસસસસીીસસસસસસસસસસસસસસસઈઈઈઈઈઈઈ૮૮સસઈઈધ૮દદદદદદ૮૮5દથથથથથથથથથથ»»૦૦…*થથથથધથધથથ’’’’’’’’’ઢઢઢ’’’’’’555થથ!!!!!!!!ૄૄૄૄૄૄ!!!!!!!!!!!!!',)
-------------------------------------------
Iteration 100 : 3.5010406970977783
English: PM visits Hubli in Karnataka
gujarati Translation: પ્રધાનમંત્રીએ કર્ણાટકમાં હુબલીની મુલાકાત લીધી
gujarati Prediction: તતેતા                                               
Evaluation translation (should we go to the mall?) : ('                                          

In [105]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [106]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [107]:
class SentenceEmbedding(nn.Module):
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
#         print(self.vocab_size)
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token, end_token):

        def tokenize(sentence, start_token, end_token):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
            tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(device)
    
    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
#         print(x.shape)
        x = self.embedding(x)
        pos = self.position_encoder().to(device)
        x = self.dropout(x + pos)
        return x

In [108]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model # 512
        self.num_heads = num_heads # 8
        self.head_dim = d_model // num_heads # 64
        self.qkv_layer = nn.Linear(d_model, 3 * d_model) # 512 -> 1536
        self.linear_layer = nn.Linear(d_model, d_model) # 512 -> 512
        
        
    def forward(self,x,mask=None):
        batch_size, seq_len, d_model = x.shape # 30 x 200 x 512
        qkv = self.qkv_layer(x) # 30 x 200 x 1536
        qkv = qkv.reshape(batch_size,seq_len,self.num_heads, 3*self.head_dim) # 30 x 200 x 8 x 192
        qkv = qkv.permute(0,2,1,3) # 30 x 8 x 200 x 192
        q, k, v = qkv.chunk(3,dim=-1) # [30 x 8 x 200 x 64] q,k,v
        values, attentions = scaled_dot_product(q, k, v, mask=None) # values = 30 x 8 x 200 x 64 , attention = 30 x 8 x 200 x 200 
        values = values.reshape(batch_size,seq_len,self.num_heads*self.head_dim) # 30 x 200 x 512
        out = self.linear_layer(values) # 30 x 200 x 512
        return out
        

In [109]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward,self).__init__()
        self.linear1 = nn.Linear(d_model,hidden) # 512 x hidden
        self.linear2 = nn.Linear(hidden,d_model) # hidden x 512
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(p=drop_prob)
        
    def forward(self,x): # 30 x 200 x 512
        x = self.linear1(x) # 30 x 512 x hidden
        x = self.relu(x) # 30 x 512 x hidden
        x = self.dropout(x) # 30 x 200 x hidden
        x = self.linear2(x) # 30 x 200 x 512
        return x

In [110]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape = parameters_shape # [512]
        self.eps = eps 
        self.gamma = nn.Parameter(torch.ones(parameters_shape)) # 512
        self.beta = nn.Parameter(torch.zeros(parameters_shape)) # 512
        
    def forward(self, inputs): # inputs = 30 x 200 x 512
        dims = [-(i+1) for i in range(len(self.parameters_shape))] # [-1]
        mean = inputs.mean(dim=dims, keepdim = True) # 30 x 200 x 1
        var = ((inputs-mean)**2).mean(dim=dims, keepdim=True) # 30 x 200 x 1
        std = (var + self.eps).sqrt() # 30 x 200 x 1
        y = (inputs-mean) / std # 30 x 200 x 512
        out = self.gamma * y + self.beta # 30 x 200 x 512
        return out

In [111]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer,self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
    
    def forward(self, x, self_attention_mask):
        residual_x = x.clone() # 30 x 200 x 512
        x = self.attention(x, mask=self_attention_mask) # 30 x 200 x 512
        x = self.dropout1(x) # 30 x 200 x 512
        x = self.norm1(x + residual_x) # 30 x 200 x 512 
        
        residual_x = x.clone() # 30 x 200 x 512
        
        x = self.ffn(x) # 30 x 200 x 512
        x = self.dropout2(x) # 30 x 200 x 512
        x = self.norm2(x + residual_x) # 30 x 200 x 512
        
        return x # 30 x 200 x 512

In [112]:
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
#         print(inputs)
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x


In [113]:
class Encoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_seq_len,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_seq_len, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x

In [114]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self,d_model,num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model, 2 * d_model)
        self.q_layer = nn.Linear(d_model, d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
        
    def forward(self, x, y, mask=None):
        batch_size, seq_len, d_model = x.shape # 30 x 200 x 512
        kv = self.kv_layer(x) # 30 x 200 x 1536
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, seq_len, self.num_heads, 2 * self.head_dim) # 30 x 200 x 8 x 192
        q = q.reshape(batch_size, seq_len, self.num_heads, self.head_dim)
        
        kv = kv.permute(0,2,1,3) # 30 x 8 x 200 x 192
        q = q.permute(0,2,1,3)
        
        k, v = kv.chunk(2,dim=-1) # [30 x 8 x 200 x 64] k,v

        values, attentions = scaled_dot_product(q, k, v, mask) # values = 30 x 8 x 200 x 64 , attention = 30 x 8 x 200 x 200 
        values = values.reshape(batch_size,seq_len,self.num_heads*self.head_dim) # 30 x 200 x 512
        out = self.linear_layer(values) # 30 x 200 x 512
        return out

In [115]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer,self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization([d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        
        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)
    
    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
#         print("Masked Self Attention")
        y = self.self_attention(x,mask=self_attention_mask)
#         print("Dropout 1")
        y = self.dropout1(y)
#         print("Add + Layer Normalization 1")
        y = self.norm1(y + _y)
        
        _y = y.clone()
#         print("Cross Attention")
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
#         print("Dropout 2")
        y = self.dropout2(y)
#         print("Add + Layer Normalization 2")
        y = self.norm2(y + _y)
        
        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.norm3(y + _y)
        return y

In [116]:
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

In [117]:
class Decoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_seq_len,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_seq_len, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

In [118]:

class Transformer(nn.Module):
    def __init__(self, 
                d_model, 
                ffn_hidden, 
                num_heads, 
                drop_prob, 
                num_layers,
                max_seq_len, 
                guj_vocab_size,
                english_to_index,
                gujarati_to_index,
                START_TOKEN, 
                END_TOKEN, 
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_seq_len, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_seq_len, gujarati_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, guj_vocab_size)

    def forward(self, 
                x, 
                y, 
                encoder_self_attention_mask=None, 
                decoder_self_attention_mask=None, 
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out